In [2]:
import pandas as pd

import wmfdata as wmf
from wmfdata.utils import (
    pd_display_all,
    df_to_remarkup
)

Recently, I updated the `wikipediapreview_stats` ETL job ([T314829](https://phabricator.wikimedia.org/T314829)). Now the newest data looks a bit suspicious and I need to check whether something went wrong.

I can already see something I missed: when I created the new external table, I checked the daily counts of pageviews and previews, but didn't notice they had exactly doubled from what they were previously, except for 2022-08-24, the first day the new job had run ([T314829#8187147](https://phabricator.wikimedia.org/T314829#8187147)).

Let's see where the data is stored in HDFS.

In [4]:
create_statement = wmf.spark.run("""
SHOW CREATE TABLE
wmf_product.wikipediapreview_stats
""").iloc[0, 0]

print(create_statement)

CREATE EXTERNAL TABLE `wmf_product`.`wikipediapreview_stats`(`pageviews` BIGINT COMMENT 'Number of pageviews shown as a result of a clickthrough from a Wikipedia Preview preview', `previews` BIGINT COMMENT 'Number of API requests for article preview content made by Wikipedia Preview clients', `year` INT COMMENT 'Unpadded year of request', `month` INT COMMENT 'Unpadded month of request', `day` INT COMMENT 'Unpadded day of request', `device_type` STRING COMMENT 'Type of device used by the client: touch or non-touch', `referer_host` STRING COMMENT 'Host from referer parsing', `continent` STRING COMMENT 'Continent of the accessing agents (maxmind GeoIP database)', `country_code` STRING COMMENT 'Country iso code of the accessing agents (maxmind GeoIP database)', `country` STRING COMMENT 'Country (text) of the accessing agents (maxmind GeoIP database)')
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'line.delim' = '
',
  'field.delim' = '	',
  

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


Unsurprisingly, that has two data files, which are getting combined when we query the data through the Hive tables.

In [5]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 2 items
-rwxrwxr-x   3 analytics-product analytics-privatedata-users    2556062 2022-08-25 21:19 /user/analytics-product/wikipediapreview_stats/daily/000000_0
-rw-r--r--   3 analytics-product hdfs                           1816379 2022-08-31 00:32 /user/analytics-product/wikipediapreview_stats/daily/data.gz


What the what? Now the data up to 2022-08-23 is multiplied by 8 times! So it seems like it not just a one-time duplication; it's probably something that's happening each time the job runs.

In [7]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
    AND device_type = "non-touch"
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,5864,48
1,2022,8,18,5176,16
2,2022,8,19,4448,24
3,2022,8,20,4448,56
4,2022,8,21,4128,40
5,2022,8,22,5904,32
6,2022,8,23,4744,32
7,2022,8,24,688,30
8,2022,8,25,677,5
9,2022,8,26,520,6


22/08/31 23:30:48 WARN UserGroupInformation: Not attempting to re-login since the last re-login was attempted less than 60 seconds before. Last Login=1661988632615
22/08/31 23:31:32 WARN UserGroupInformation: Exception encountered while running the renewal command for neilpquinn-wmf@WIKIMEDIA. (TGT end time:1661988653000, renewalFailures: org.apache.hadoop.metrics2.lib.MutableGaugeInt@3722280,renewalFailuresTotal: org.apache.hadoop.metrics2.lib.MutableGaugeLong@17639676)
ExitCodeException exitCode=1: kinit: Ticket expired while renewing credentials

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:998)
	at org.apache.hadoop.util.Shell.run(Shell.java:884)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1216)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1310)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1292)
	at org.apache.hadoop.security.UserGroupInformation$1.run(UserGroupInformation.java:1003)
	at java.lang.Thread.run(Thread.java:7

Looking at the job code, the job expects the data to be in `data.gz`. That reinforces my sense that the problem is the other file. Let me try moving it somewhere else and seeing if that fixes the data.

In [12]:
!hdfs dfs -cp /user/analytics-product/wikipediapreview_stats/daily/000000_0 /user/neilpquinn-wmf

I can't run sudo commands in the Jupyter environment, so I'll just document it here:
```
sudo -u analytics-product kerberos-run-command analytics-product hdfs dfs -rm /user/analytics-product/wikipediapreview_stats/daily/000000_0
```

In [14]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 1 items
-rw-r--r--   3 analytics-product hdfs    1816379 2022-08-31 00:32 /user/analytics-product/wikipediapreview_stats/daily/data.gz


Hmm, now it's only duplicated 7 times! But I do think that removes the source of the daily duplication. Now I just need to remove the previously duplicated data.

In [13]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
    AND device_type = "non-touch"
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,5131,42
1,2022,8,18,4529,14
2,2022,8,19,3892,21
3,2022,8,20,3892,49
4,2022,8,21,3612,35
5,2022,8,22,5166,28
6,2022,8,23,4151,28
7,2022,8,24,688,30
8,2022,8,25,677,5
9,2022,8,26,520,6


Also, all this time I've just been querying for non-touch events. I should switch to querying all events just so that I'm not missing anything.

In [15]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,5670,455
1,2022,8,18,4921,343
2,2022,8,19,4312,259
3,2022,8,20,4221,371
4,2022,8,21,3920,385
5,2022,8,22,5467,273
6,2022,8,23,4438,413
7,2022,8,24,726,65
8,2022,8,25,721,47
9,2022,8,26,563,65


And the backed-up data for comparison:

In [16]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM nshahquinn.wikipediapreview_stats_backup
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,810,65
1,2022,8,18,703,49
2,2022,8,19,616,37
3,2022,8,20,603,53
4,2022,8,21,560,55
5,2022,8,22,781,39
6,2022,8,23,634,59


Okay, so let me delete all the duplicated data and replace it with data from the backup. First I need to verify how to select the data I want to delete.

In [22]:
wmf.spark.run("""
SELECT COUNT(*)
FROM wmf_product.wikipediapreview_stats
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,count(1)
0,289675


In [25]:
wmf.spark.run("""
SELECT COUNT(*)
FROM wmf_product.wikipediapreview_stats
WHERE
    (
        year < 2022
        OR year = 2022 AND month < 8
        OR year = 2022 AND month = 8 AND day < 24
    )
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,count(1)
0,288568


Okay, now to actually delete.

In [26]:
wmf.spark.run("""
DELETE
FROM wmf_product.wikipediapreview_stats
WHERE
    (
        year < 2022
        OR year = 2022 AND month < 8
        OR year = 2022 AND month = 8 AND day < 24
    )
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


ParseException: '\nOperation not allowed: DELETE FROM(line 2, pos 0)\n\n== SQL ==\n\nDELETE\n^^^\nFROM wmf_product.wikipediapreview_stats\nWHERE\n    (\n        year < 2022\n        OR year = 2022 AND month < 8\n        OR year = 2022 AND month = 8 AND day < 24\n    )\n'

Can I delete with Hive, then?

In [27]:
wmf.hive.run("""
DELETE
FROM wmf_product.wikipediapreview_stats
WHERE
    (
        year < 2022
        OR year = 2022 AND month < 8
        OR year = 2022 AND month = 8 AND day < 24
    )
""")

DatabaseError: Execution failed on sql: 
DELETE
FROM wmf_product.wikipediapreview_stats
WHERE
    (
        year < 2022
        OR year = 2022 AND month < 8
        OR year = 2022 AND month = 8 AND day < 24
    )

TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['*org.apache.hive.service.cli.HiveSQLException:Error while compiling statement: FAILED: SemanticException [Error 10294]: Attempt to do update or delete using transaction manager that does not support these operations.:28:27', 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:380', 'org.apache.hive.service.cli.operation.SQLOperation:prepare:SQLOperation.java:206', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:290', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:320', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:530', 'org.apache.hive.service.cli.session.HiveSessionImpl:execut

Now that the job has done a couple more daily runs, it's clear that we've stopped adding new duplicates, so that extra file was the root cause.

In [28]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/09/01 23:10:31 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/09/01 23:10:31 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/09/01 23:10:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/01 23:10:37 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
22/09/01 23:10:37 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


,year,month,day,previews,pageviews
0,2022,8,17,5670,455
1,2022,8,18,4921,343
2,2022,8,19,4312,259
3,2022,8,20,4221,371
4,2022,8,21,3920,385
5,2022,8,22,5467,273
6,2022,8,23,4438,413
7,2022,8,24,726,65
8,2022,8,25,721,47
9,2022,8,26,563,65


Now to clean the old duplicates up.

In [30]:
wmf.spark.run("""
CREATE TABLE nshahquinn.wikipediapreview_stats_combined
LIKE wmf_product.wikipediapreview_stats
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


wmf.spark.run("""
INSERT INTO nshahquinn.wikipediapreview_stats_combined
SELECT
    pageviews,
    previews,
    year,
    month,
    day,
    CASE access_method
        WHEN 'mobile web' THEN 'touch'
        WHEN 'desktop' THEN 'non-touch'
    END AS device_type,
    referer_host,
    continent,
    country_code,
    country
FROM nshahquinn.wikipediapreview_stats_backup
""")

In [33]:
wmf.spark.run("""
INSERT INTO nshahquinn.wikipediapreview_stats_combined
SELECT *
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 24
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


The counts in the combined dataset look right now.

In [34]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM nshahquinn.wikipediapreview_stats_combined
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,810,65
1,2022,8,18,703,49
2,2022,8,19,616,37
3,2022,8,20,603,53
4,2022,8,21,560,55
5,2022,8,22,781,39
6,2022,8,23,634,59
7,2022,8,24,726,65
8,2022,8,25,721,47
9,2022,8,26,563,65


The non-touch counts (which I was checking before) look right too and match what I had previously.

In [37]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM nshahquinn.wikipediapreview_stats_combined
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
    AND device_type = "non-touch"
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,733,6
1,2022,8,18,647,2
2,2022,8,19,556,3
3,2022,8,20,556,7
4,2022,8,21,516,5
5,2022,8,22,738,4
6,2022,8,23,593,4
7,2022,8,24,688,30
8,2022,8,25,677,5
9,2022,8,26,520,6


Now to replace the existing data with the combined data. I don't need to drop the table since I can delete the underlying data in HDFS.

In [49]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 1 items
-rw-r--r--   3 analytics-product hdfs    1817587 2022-09-01 00:39 /user/analytics-product/wikipediapreview_stats/daily/data.gz


`sudo -u analytics-product kerberos-run-command analytics-product hdfs dfs -rm /user/analytics-product/wikipediapreview_stats/daily/data.gz`

In [51]:
wmf.spark.run("""
INSERT INTO wmf_product.wikipediapreview_stats
SELECT *
FROM nshahquinn.wikipediapreview_stats_combined
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


Hmm, it inserted the data as separate files, so I can't just rename it to `data.gz`.

In [52]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 3 items
-rwxrwxr-x   3 neilpquinn-wmf analytics-privatedata-users    1280434 2022-09-01 23:53 /user/analytics-product/wikipediapreview_stats/daily/part-00000-6be8280d-2794-4258-9e09-b1fc354c5557-c000
-rwxrwxr-x   3 neilpquinn-wmf analytics-privatedata-users      80071 2022-09-01 23:53 /user/analytics-product/wikipediapreview_stats/daily/part-00001-6be8280d-2794-4258-9e09-b1fc354c5557-c000
-rwxrwxr-x   3 neilpquinn-wmf analytics-privatedata-users    1275628 2022-09-01 23:53 /user/analytics-product/wikipediapreview_stats/daily/part-00002-6be8280d-2794-4258-9e09-b1fc354c5557-c000


At least the data is right now.

In [55]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,810,65
1,2022,8,18,703,49
2,2022,8,19,616,37
3,2022,8,20,603,53
4,2022,8,21,560,55
5,2022,8,22,781,39
6,2022,8,23,634,59
7,2022,8,24,726,65
8,2022,8,25,721,47
9,2022,8,26,563,65


Also, while I'm at it, let me kill the job so I don't get another run happening while I'm doing this.

`oozie job -kill 0079924-220613130955581-oozie-oozi-C`

In [60]:
!hdfs dfs -rm /user/analytics-product/wikipediapreview_stats/daily/*

22/09/02 00:26:21 INFO fs.TrashPolicyDefault: Moved: 'hdfs://analytics-hadoop/user/analytics-product/wikipediapreview_stats/daily/000000_0' to trash at: hdfs://analytics-hadoop/user/neilpquinn-wmf/.Trash/Current/user/analytics-product/wikipediapreview_stats/daily/000000_0


According to [this StackOverflow answer](https://stackoverflow.com/questions/56593175/is-there-a-way-to-merge-orc-files-in-hdfs-without-using-alter-table-concatenate/56596869#56596869), I can force Hive to output a single data file by including an ORDER BY clause. I also want it to GZIP the output.

In [61]:
wmf.hive.run([
    "SET hive.exec.compress.output=true",
    "SET mapreduce.output.fileoutputformat.compress.codec=org.apache.hadoop.io.compress.GzipCodec",
    """
    INSERT INTO wmf_product.wikipediapreview_stats
    SELECT *
    FROM nshahquinn.wikipediapreview_stats_combined
    ORDER BY
        year,
        month,
        day
    LIMIT 1000000
    """
])

Success!

In [62]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 1 items
-rwxrwxr-x   3 neilpquinn-wmf analytics-privatedata-users     261170 2022-09-02 00:27 /user/analytics-product/wikipediapreview_stats/daily/000000_0.gz


In [65]:
!hdfs dfs -mv /user/analytics-product/wikipediapreview_stats/daily/000000_0.gz /user/analytics-product/wikipediapreview_stats/daily/data.gz 

Great, now I have one data file named `data.gz`

In [66]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 1 items
-rwxrwxr-x   3 neilpquinn-wmf analytics-privatedata-users     261170 2022-09-02 00:27 /user/analytics-product/wikipediapreview_stats/daily/data.gz


And the counts are right!

In [67]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 8
    AND day >= 17
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,810,65
1,2022,8,18,703,49
2,2022,8,19,616,37
3,2022,8,20,603,53
4,2022,8,21,560,55
5,2022,8,22,781,39
6,2022,8,23,634,59
7,2022,8,24,726,65
8,2022,8,25,721,47
9,2022,8,26,563,65


Redeployed the job starting from 2022-09-01.

```
neilpquinn-wmf@stat1005:~/product_analytics_jobs$ ./deploy-oozie-job wikipediapreview_stats --production
The HDFS job directory will be hdfs:///user/analytics-product/jobs/wikipediapreview_stats
Removing old job files in the job directory...
Creating the job directory...
Putting new job files into the job directory...
Submitting the job...
job: 0087801-220613130955581-oozie-oozi-C
```

And the new run worked fine! I think this is finally fixed.

In [71]:
wmf.spark.run("""
SELECT
    year,
    month,
    day,
    SUM(previews) AS previews,
    SUM(pageviews) AS pageviews
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND (
        month = 8 AND day >= 17
        OR month = 9
    )
GROUP BY
    year,
    month,
    day
ORDER BY
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,day,previews,pageviews
0,2022,8,17,810,65
1,2022,8,18,703,49
2,2022,8,19,616,37
3,2022,8,20,603,53
4,2022,8,21,560,55
5,2022,8,22,781,39
6,2022,8,23,634,59
7,2022,8,24,726,65
8,2022,8,25,721,47
9,2022,8,26,563,65


In [72]:
!hdfs dfs -ls /user/analytics-product/wikipediapreview_stats/daily

Found 1 items
-rw-r--r--   3 analytics-product hdfs     262375 2022-09-02 00:43 /user/analytics-product/wikipediapreview_stats/daily/data.gz
